### Work In Progress

In [1]:
import tweepy
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import re
import string
nltk.download('stopwords')

consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [98]:
def get_clean_tweets(query, max_tweets):
    searched_tweets = [tweet for tweet in tweepy.Cursor(api.search, q=query, lang='en').items(max_tweets)]
    clean_tweets = []
    emoji_uni = list(emoji_dict.keys())
    for tweet in searched_tweets:
        tweet_text = tweet.text 
        tweet_low = tweet_text.lower()  
        tweet_link = re.sub(r'https\S+', '', tweet_low) 
        clean_punc = re.sub(r"[,.:;'\"’“”@#%-?!&$]+\ *", '', tweet_link)
        clean_rest = re.sub('rt|…', '', clean_punc)
        tokenizer = TweetTokenizer()
        tokens = tokenizer.tokenize(clean_rest)
        clean_stop_words = [w for w in tokens if not w in stopwords.words('english')]
        clean_emojis = [x for x in clean_stop_words if not x in emoji_uni]
        clean_tweets.append(clean_emojis)
    return clean_tweets

In [99]:
data = get_clean_tweets('craftbeer', 50)
print(data[:50])

[['hopnationusawrestlemania', 'todayfind', 'beers', 'best', 'embody', 'wwe', 'nxt', 'superstars', 'click'], ['corona', 'limeits', 'fine', 'logoff', 'amplimein', 'extra', 'beer', '🍻', '🍺', '🍹', 'drink', 'drinks', 'slurp', 'toptags', 'pub'], ['hey', 'craftbrew', 'enthusiastsoutdoors', 'peopletravelersnext', 'time', 'go', 'jasper', 'strongly', 'recommend', 'stop'], ['cheers', 'sundayfunday', 'drinklocal', 'craftbeer', 'kokopellibeerco'], ['sunday', 'weve', 'got', 'free', 'live', 'music', 'brewery', 'tonightcome', 'join', 'us', 'pm', 'enjoy', 'musical', 'eveni'], ['dubcentralreally', 'enjoyed', 'lemon', 'sour', 'thewhitehag', 'picked', 'brilliant', 'weebeershop', 'glasgow', 'craftbeer'], ['mommycampbelli', 'pose', 'flights', 'wife', 'thinks', 'im', 'dorkbut', 'think', 'im', 'cool', 'rightnationalbeerday', 'still', 'going', 'fo'], ['npattersonafter', 'couple', 'weeks', 'aging', 'last', 'postbrown', 'ale', 'headed', 'brite', 'tank', 'one', 'day', 'closer', 'making'], ['stonebrewing', 'pa', '

In [100]:
import collections

data = [item for sublist in data for item in sublist]
counter=collections.Counter(data)
sorted_words = sorted(counter.items(), key=lambda x:x[1], reverse=True)
sorted_words[:25]

[('craftbeer', 21),
 ('im', 8),
 ('beer', 5),
 ('sunday', 4),
 ('lemon', 4),
 ('pose', 4),
 ('flights', 4),
 ('wife', 4),
 ('thinks', 4),
 ('dorkbut', 4),
 ('think', 4),
 ('cool', 4),
 ('rightnationalbeerday', 4),
 ('still', 4),
 ('going', 4),
 ('ipa', 4),
 ('drinking', 4),
 ('enjoyed', 3),
 ('sour', 3),
 ('thewhitehag', 3),
 ('picked', 3),
 ('brilliant', 3),
 ('weebeershop', 3),
 ('glasgow', 3),
 ('mommycampbelli', 3)]